In [4]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder \
    .master("spark://spark-master:7077") \
    .appName("JupyterSparkCluster") \
    .getOrCreate()

# Verify the Spark Context
print(spark)